In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.layers import Dense,Flatten,Input,Lambda,Dropout
from tensorflow.keras.models import Model 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


In [ ]:
!nvidia-smi

Fri Apr 30 03:09:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Data Augumentation 
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   zoom_range=0.2,
                                   height_shift_range = 0.2,
                                   width_shift_range = 0.2)

# Note that validation data should not be augumented 
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory("/content/drive/MyDrive/Skin_Cancer_train_data",
                                                    target_size=(224,224),
                                                    batch_size=20,
                                                    class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory("/content/drive/MyDrive/Skin_Cancer_test_data",
                                                  batch_size=20,
                                                  class_mode='binary',
                                                  target_size=(224,224)
                                                  )

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [ ]:
# Loading the base model 
from tensorflow.keras.applications.vgg16 import VGG16
Image_Size = [224,224]
base_model = VGG16(input_shape = Image_Size + [3], # shape of our image
                   weights='imagenet',
                   include_top = False) # Leave out the last fully connected layer

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
#Since we don’t have to train all the layers, we make them non_trainable:
for layer in base_model.layers:
  layer.trainable = False


In [ ]:
# Step 4: Compile and Fit
# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512,activation='relu')(x)

# Add a dropout rate of 0.2
x= Dropout(0.2)(x)

# Add a final sigmoid layer for classification
x= Dense(1,activation='sigmoid')(x)

model = Model(base_model.input, x)


In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
# compile
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#fit
model.fit(train_generator,validation_data=test_generator,steps_per_epoch=100,epochs=10)

Epoch 1/10
100/100 [==============================] - 653s 6s/step - loss: 1.5270 - accuracy: 0.6253 - val_loss: 0.3749 - val_accuracy: 0.8227
Epoch 2/10
100/100 [==============================] - 129s 1s/step - loss: 0.4926 - accuracy: 0.7631 - val_loss: 0.3688 - val_accuracy: 0.8364
Epoch 3/10
100/100 [==============================] - 57s 569ms/step - loss: 0.4146 - accuracy: 0.8052 - val_loss: 0.3397 - val_accuracy: 0.8394
Epoch 4/10
100/100 [==============================] - 35s 348ms/step - loss: 0.3772 - accuracy: 0.8196 - val_loss: 0.3385 - val_accuracy: 0.8394
Epoch 5/10
100/100 [==============================] - 33s 331ms/step - loss: 0.3503 - accuracy: 0.8384 - val_loss: 0.3498 - val_accuracy: 0.8333
Epoch 6/10
100/100 [==============================] - 31s 306ms/step - loss: 0.3599 - accuracy: 0.8187 - val_loss: 0.3349 - val_accuracy: 0.8409
Epoch 7/10
100/100 [==============================] - 31s 307ms/step - loss: 0.3510 - accuracy: 0.8334 - val_loss: 0.3718 - val_accura